In [28]:
import os

import b3d
import b3d.bayes3d as bayes3d
import fire
import genjax
import jax
import jax.numpy as jnp
import numpy as np
import rerun as rr
from b3d import Pose
from genjax import Pytree
from tqdm import tqdm
genjax.pretty()

In [2]:
rr.init("demo")
rr.connect("127.0.0.1:8812")

In [3]:
path = os.path.join(
    b3d.get_root_path(),
    "assets/shared_data_bucket/input_data/shout_on_desk.r3d.video_input.npz",
)
video_input = b3d.io.VideoInput.load(path)

# Get intrinsics
image_width, image_height, fx, fy, cx, cy, near, far = np.array(
    video_input.camera_intrinsics_depth
)
image_width, image_height = int(image_width), int(image_height)
fx, fy, cx, cy, near, far = (
    float(fx),
    float(fy),
    float(cx),
    float(cy),
    float(near),
    float(far),
)

# Get RGBS and Depth
rgbs = video_input.rgb[::4] / 255.0
xyzs = video_input.xyz[::4]

In [4]:
image_width, image_height

(192, 256)

In [5]:
fx, fy, cx, cy, near, far

(181.88333129882812,
 181.88333129882812,
 96.39157104492188,
 128.85830688476562,
 0.009999999776482582,
 100.0)

In [6]:
rgbs.shape

(186, 960, 720, 3)

In [7]:
xyzs.shape

(186, 256, 192, 3)

In [8]:
rgbs_resized = jnp.clip(
    jax.vmap(jax.image.resize, in_axes=(0, None, None))(
        rgbs, (video_input.xyz.shape[1], video_input.xyz.shape[2], 3), "linear"
    ),
    0.0,
    1.0,
)
rgbs_resized.shape

(186, 256, 192, 3)

In [9]:
# Make empty library
object_library = bayes3d.MeshLibrary.make_empty_library()

# Take point cloud at frame 0
point_cloud = jax.image.resize(
    xyzs[0], (xyzs[0].shape[0] // 3, xyzs[0].shape[1] // 3, 3), "linear"
).reshape(-1, 3)
colors = jax.image.resize(
    rgbs_resized[0], (xyzs[0].shape[0] // 3, xyzs[0].shape[1] // 3, 3), "linear"
).reshape(-1, 3)

# `make_mesh_from_point_cloud_and_resolution` takes a 3D positions, colors, and sizes of the boxes that we want
# to place at each position and create a mesh
vertices, faces, vertex_colors, _face_colors = (
    b3d.make_mesh_from_point_cloud_and_resolution(
        point_cloud,
        colors,
        point_cloud[:, 2]
        / fx
        * 3.0,  # This is scaling the size of the box to correspond to the effective size of the pixel in 3D. It really should be multiplied by 2.
        # and the 6 makes it larger
    )
)
object_library.add_object(vertices, faces, vertex_colors)

In [10]:
vertices.shape

(130560, 3)

In [11]:
vertex_colors.shape

(130560, 3)

In [12]:
# Gridding on translation only.
translation_deltas = Pose.concatenate_poses(
    [
        jax.vmap(lambda p: Pose.from_translation(p))(
            jnp.stack(
                jnp.meshgrid(
                    jnp.linspace(-0.01, 0.01, 11),
                    jnp.linspace(-0.01, 0.01, 11),
                    jnp.linspace(-0.01, 0.01, 11),
                ),
                axis=-1,
            ).reshape(-1, 3)
        ),
        Pose.identity()[None, ...],
    ]
)
# Sample orientations from a VMF to define a "grid" over orientations.
rotation_deltas = Pose.concatenate_poses(
    [
        jax.vmap(Pose.sample_gaussian_vmf_pose, in_axes=(0, None, None, None))(
            jax.random.split(jax.random.PRNGKey(0), 11 * 11 * 11),
            Pose.identity(),
            0.00001,
            1000.0,
        ),
        Pose.identity()[None, ...],
    ]
)
all_deltas = Pose.stack_poses([translation_deltas, rotation_deltas])


In [13]:
all_deltas

Pose(position=Array([[[-9.9999998e-03, -9.9999998e-03, -9.9999998e-03],
        [-9.9999998e-03, -9.9999998e-03, -7.9999994e-03],
        [-9.9999998e-03, -9.9999998e-03, -6.0000001e-03],
        ...,
        [ 9.9999998e-03,  9.9999998e-03,  8.0000013e-03],
        [ 9.9999998e-03,  9.9999998e-03,  9.9999998e-03],
        [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00]],

       [[ 3.5493358e-06,  2.4435249e-06, -1.0975113e-05],
        [-1.3543133e-05, -7.1426111e-06, -1.3748184e-05],
        [ 1.0893336e-06,  7.7870573e-07,  5.0377867e-07],
        ...,
        [-8.8171946e-06, -7.5722419e-06,  1.1219221e-05],
        [-1.2537406e-05, -1.2625285e-05, -1.9125891e-06],
        [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00]]], dtype=float32), quaternion=Array([[[ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  1.0000000e+00],
        [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  1.0000000e+00],
        [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  1.0000000e+00],
        ...,
        [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  1.0000000e+00],
        [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  1.0000000e+00],
        [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  1.0000000e+00]],

       [[-3.0204833e-02, -2.9932808e-02, -5.8132381e-04,  9.9909526e-01],
        [ 2.1666827e-02,  1.7800909e-02, -9.0586161e-03,  9.9956578e-01],
        [-2.5553171e-02, -3.3522133e-02, -3.4941919e-02,  9.9850011e-01],
        ...,
        [-4.2501693e-03, -5.3908288e-02,  3.1391408e-02,  9.9804336e-01],
        [ 3.5580162e-02,  9.2606451e-03,  3.5366002e-02,  9.9869800e-01],
        [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  1.0000000e+00]]],      dtype=float32))

In [14]:
all_deltas[1]

Pose(position=Array([[ 3.5493358e-06,  2.4435249e-06, -1.0975113e-05],
       [-1.3543133e-05, -7.1426111e-06, -1.3748184e-05],
       [ 1.0893336e-06,  7.7870573e-07,  5.0377867e-07],
       ...,
       [-8.8171946e-06, -7.5722419e-06,  1.1219221e-05],
       [-1.2537406e-05, -1.2625285e-05, -1.9125891e-06],
       [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00]], dtype=float32), quaternion=Array([[-3.0204833e-02, -2.9932808e-02, -5.8132381e-04,  9.9909526e-01],
       [ 2.1666827e-02,  1.7800909e-02, -9.0586161e-03,  9.9956578e-01],
       [-2.5553171e-02, -3.3522133e-02, -3.4941919e-02,  9.9850011e-01],
       ...,
       [-4.2501693e-03, -5.3908288e-02,  3.1391408e-02,  9.9804336e-01],
       [ 3.5580162e-02,  9.2606451e-03,  3.5366002e-02,  9.9869800e-01],
       [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  1.0000000e+00]],      dtype=float32))

In [15]:
# Defines the enumeration schedule.
key = jax.random.PRNGKey(0)
renderer = b3d.Renderer(image_width, image_height, fx, fy, cx, cy, near, far)
model = bayes3d.model_multiobject_gl_factory(renderer)
importance_jit = jax.jit(model.importance)

# Arguments of the generative model.
# These control the inlier / outlier decision boundary for color error and depth error.
color_error, depth_error = (60.0, 0.01)
inlier_score, outlier_prob = (5.0, 0.00001)
color_multiplier, depth_multiplier = (10000.0, 500.0)
model_args = bayes3d.ModelArgs(
    color_error,
    depth_error,
    inlier_score,
    outlier_prob,
    color_multiplier,
    depth_multiplier,
)

/home/hlwang_ipe_genjax/b3d/.pixi/envs/gpu/lib/python3.12/site-packages/torch/utils/cpp_extension.py:1967: UserWarning: TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'].
  warnings.warn(


In [16]:
# Initial trace for timestep 0
START_T = 0
trace, _ = importance_jit(
    jax.random.PRNGKey(0),
    genjax.ChoiceMap.d(
        dict(
            [
                ("camera_pose", Pose.identity()),
                ("object_pose_0", Pose.identity()),
                ("object_0", 0),
                (
                    "observed_rgb_depth",
                    (rgbs_resized[START_T], xyzs[START_T, ..., 2]),
                ),
            ]
        )
    ),
    (jnp.arange(1), model_args, object_library),
)

In [17]:
trace

StaticTrace(...)

In [18]:
trace.get_retval()

((<jax.Array float32(256, 192, 3) ≈0.58 ±0.24 [≥0.0039, ≤0.99] nonzero:147_456
    <Arrayviz rendering>
  >,
  <jax.Array float32(256, 192, 3) ≈0.58 ±0.23 [≥0.0, ≤0.96] zero:12 nonzero:147_444
    <Arrayviz rendering>
  >),
 (<jax.Array float32(256, 192) ≈0.51 ±0.18 [≥0.24, ≤0.86] nonzero:49_152
    <Arrayviz rendering>
  >,
  <jax.Array float32(256, 192) ≈0.51 ±0.18 [≥0.0, ≤0.85] zero:4 nonzero:49_148
    <Arrayviz rendering>
  >))

In [19]:
# Visualize trace
bayes3d.rerun_visualize_trace_t(trace, 0)


In [20]:
ACQUISITION_T = 90
for T_observed_image in tqdm(range(ACQUISITION_T)):
    # Constrain on new RGB and Depth data.
    trace = b3d.update_choices(
        trace,
        Pytree.const(["observed_rgb_depth"]),
        (rgbs_resized[T_observed_image], xyzs[T_observed_image, ..., 2]),
    )
    trace, key = bayes3d.enumerate_and_select_best_move(
        trace, Pytree.const(("camera_pose",)), key, all_deltas
    )
    bayes3d.rerun_visualize_trace_t(trace, T_observed_image)


100%|██████████| 90/90 [00:42<00:00,  2.12it/s]


In [21]:
# Outliers are AND of the RGB and Depth outlier masks
_inliers, _color_inliers, _depth_inliers, outliers, _undecided, _valid_data_mask = (
    bayes3d.get_rgb_depth_inliers_from_trace(trace)
)
outlier_mask = outliers
rr.log("outliers", rr.Image(jnp.tile((outlier_mask * 1.0)[..., None], (1, 1, 3))))

# Get the point cloud corresponding to the outliers
rgb, depth = trace.get_choices()["observed_rgb_depth"]
point_cloud = b3d.xyz_from_depth(depth, fx, fy, cx, cy)[outlier_mask]
point_cloud_colors = rgb[outlier_mask]

# Segment the outlier cloud.
assignment = b3d.segment_point_cloud(point_cloud)

# Only keep the largers cluster in the outlier cloud.
point_cloud = point_cloud.reshape(-1, 3)[assignment == 0]
point_cloud_colors = point_cloud_colors.reshape(-1, 3)[assignment == 0]

# Create new mesh.
vertices, faces, vertex_colors, _face_colors = (
    b3d.make_mesh_from_point_cloud_and_resolution(
        point_cloud, point_cloud_colors, point_cloud[:, 2] / fx * 2.0
    )
)
object_pose = Pose.from_translation(vertices.mean(0))
vertices = object_pose.inverse().apply(vertices)
object_library.add_object(vertices, faces, vertex_colors)

In [22]:
object_library

In [23]:
object_library.ranges

<jax.Array int32(2, 2) [≥0, ≤129_504] zero:1 nonzero:3
  <Arrayviz rendering>
>

In [24]:
len(object_library.ranges)

2

In [25]:
single_object_trace = trace
trace = single_object_trace

trace, _ = importance_jit(
    jax.random.PRNGKey(0),
    genjax.ChoiceMap.d(
        dict(
            [
                ("camera_pose", trace.get_choices()["camera_pose"]),
                ("object_pose_0", trace.get_choices()["object_pose_0"]),
                ("object_pose_1", trace.get_choices()["camera_pose"] @ object_pose),
                ("object_0", 0),
                ("object_1", 1),
                (
                    "observed_rgb_depth",
                    (rgbs_resized[ACQUISITION_T], xyzs[ACQUISITION_T, ..., 2]),
                ),
            ]
        )
    ),
    (jnp.arange(2), model_args, object_library),
)

In [26]:
# Visualize trace
bayes3d.rerun_visualize_trace_t(trace, ACQUISITION_T)

In [27]:
FINAL_T = len(xyzs)
for T_observed_image in tqdm(range(ACQUISITION_T, FINAL_T)):
    # Constrain on new RGB and Depth data.
    trace = b3d.update_choices(
        trace,
        Pytree.const(("observed_rgb_depth",)),
        (rgbs_resized[T_observed_image], xyzs[T_observed_image, ..., 2]),
    )
    trace, key = bayes3d.enumerate_and_select_best_move(
        trace, Pytree.const(("camera_pose",)), key, all_deltas
    )
    trace, key = bayes3d.enumerate_and_select_best_move(
        trace, Pytree.const(("object_pose_1",)), key, all_deltas
    )
    bayes3d.rerun_visualize_trace_t(trace, T_observed_image)


100%|██████████| 96/96 [02:01<00:00,  1.26s/it]
